In [8]:

# 
rc = 2
# Scaling of the model 
a = 1.2


# Height of a stitch 
H = 0.5
# Width of a stitch
W = 0.5       



##########



import numpy as np
from matplotlib import pyplot as plt

# Intrinsic radius R = l*H < a*(rc+rc^(4-1)/(4-1))
lmax = a*(rc+rc**(4-1)/(4-1))/H
lmax = (np.round(lmax)).astype(int)

# Solve r as function of R = l*H
k = np.zeros(1)
r = np.zeros(lmax)

for l in range(1, lmax + 1):
    # Create the polynomial vector
    p = np.concatenate(([1 / (4 - 1)], k, [1], [-l * H / a]))
    # Find the roots.
    roots1 = np.roots(p) 
    # Choose real roots
    real_roots = roots1[np.isreal(roots1)].real 
    # Choose positive roots
    positive_roots = real_roots[real_roots > 0] 
    # Store the positive roots
    r[l - 1] = positive_roots[0]
    
    
rmin = 3 ** 0.5
Rmin = r[r > rmin]
lmin = r.size - Rmin.size + 1    
    

# Calculate the arc length for the part before intersection 
L = np.zeros(lmin-2)
Lstitches = np.zeros(lmin-2)

for l in range(1, lmin-1):
    # Arc length 
    L[l - 1] = 2 * np.pi * a * (r[l - 1] + r[l - 1]**(4 - 1))
    Lstitches[l - 1] = L[l - 1] / W
    # Number of stiches needed
    Lstitches[l - 1] = np.round(Lstitches[l - 1])


# The last non intersecting round should also be dividible by 4
l=lmin-1
# arc length  
L0 = 2*np.pi*a*(r[l-1]+r[l-1]**(4-1))                
Lstitches0 = L0/W
# number of stiches needed per quarter
Lstitches0 =  round(Lstitches0/4)     

LstitchesBI = list(Lstitches) + [4*Lstitches0]

# How many stiches needs to be added compared to the previous row
LstitchesAddBI = np.zeros(lmin-2)

for l in range(2, lmin):
    LstitchesAddBI[l - 2] = LstitchesBI[l - 1] - LstitchesBI[l - 2]
    
    
# Display the stich count and added stitches for nonintersecting rounds. 
XX = [None] * (lmin)

for l in range(2, lmin):
    XX[l] = f"{l-1}. +{LstitchesAddBI[l-2]} ({LstitchesBI[l-1]})"
    
XX[1] = f"0. {Lstitches[0]} stitches in a magic loop"

for line in XX:
    if line is not None:
        print(line)

print()
print('Divide into four sections all with', Lstitches0, 'stitches')


# Moving to the intersecting part

# Find the first time the surface self intersects. 
rmin = (4-1)**(1/(4-2))
Rmin = r[r>rmin]

lmin = r.shape[0]-Rmin.shape[0]+1  


# Calculate the arc length for 'inner' and 'outer' section
L1 = np.zeros(lmax)
Lstitches1 = np.zeros(lmax)

L2 = np.zeros(lmax)
Lstitches2 = np.zeros(lmax)

Lstitches1[lmin - 2] = Lstitches0

for l in range(lmin-1, lmax):
    # Angle of intersection
    th1 = np.arccos(3**0.5 / 2 * (1 + r[l]**2)**0.5 / r[l])  

    # Calculate arc length for non-intersecting part (L1)
    L1[l] = (np.pi/2 - 2 * th1) * a * (r[l] + r[l]**(4 - 1))
    Lstitches1[l] = round(L1[l] / W)

    # Calculate arc length for the intersection part (L2)
    L2[l] = 2 * th1 * a * (r[l] + r[l]**(4 - 1))
    Lstitches2[l] = round(L2[l] / W)


# Number of stitches per round in the 4 quarters model 
Lstitches = np.zeros(lmax - lmin + 1)

# The first value is the number of stitches on the last non-intersecting round
Lstitches[0] = 4 * Lstitches0

Lstitches = 4 * (Lstitches1 + Lstitches2)
Lstitches = Lstitches[lmin-2:]

# Number of added stitches per round
LstitchesAdd = Lstitches[1:]-Lstitches[:-1]


# Number of traditional increases for an inner section (from accumulated increases)
# Calculate first the number of increases per inner section per round (no accumulation)
AddIn = Lstitches2[lmin - 1:] * LstitchesAdd / Lstitches[1:]
AddInSum = np.zeros(lmax - lmin + 1)

for l in range(1, lmax - lmin + 2):
    AddInSum[l - 1] = round(sum(AddIn[:l]))

AddInFinal = np.concatenate(([round(AddIn[0])], AddInSum[1:] - AddInSum[:-1]))

# Number of outer stitches moved to inner section
MoveIn = Lstitches2[lmin - 1:] - Lstitches2[lmin - 2:-1] - np.array(AddInFinal)


# Number of increases for an outer section
AddOut = LstitchesAdd / 4 - AddInFinal

AddOut1 = Lstitches1[lmin - 1:] * LstitchesAdd / Lstitches[1:]
AddOutSum = np.zeros(lmax - lmin + 1)

for l in range(1, lmax - lmin + 2):
    AddOutSum[l - 1] = round(sum(AddOut1[:l]))

AddOutFinal = np.concatenate(([round(AddOut1[0])], AddOutSum[1:] - AddOutSum[:-1]))

LstitchSum = sum(LstitchesBI)+sum(Lstitches[1:])

# Display the number of stitches per inner and outer section and number of increases
aa = (Lstitches2[lmin-1:]).astype(int)
bb = (MoveIn).astype(int)
cc = (AddInFinal).astype(int)
dd = (Lstitches1[lmin-1:]).astype(int)
ee = (AddOutFinal).astype(int)


# Create a format string to right-align the values
format_string = "{:>5}"  # This aligns the values to the right with a width of 5 characters

# Format and print the lists
formatted_aa = "".join(format_string.format(x) for x in aa)
formatted_bb = "".join(format_string.format(x) for x in bb)
formatted_cc = "".join(format_string.format(x) for x in cc)
formatted_dd = "".join(format_string.format(x) for x in dd)
formatted_ee = "".join(format_string.format(x) for x in ee)

print()
print('Number of stitches per inner section')
print(formatted_aa)
print('Number of stitches moved in')
print(formatted_bb)
print('Increases per inner section')
print(formatted_cc)
print()
print('Number of stitches per outer section')
print(formatted_dd)
print('Increases per outer section')
print(formatted_ee)
print()
print('Number of stitches in the model', (LstitchSum).astype(int))



#Plot the model

#from mpl_toolkits.mplot3d import Axes3D
%matplotlib tk


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=20, azim=74)

rd = lmax
th = np.linspace(0, 2 * np.pi, 100)
rr = np.linspace(0, r[lmax-1], rd)
rr, th = np.meshgrid(rr, th)


xx = a * (rr * np.cos(th) - rr ** 3 / 3 * np.cos(3 * th))
yy = a * (rr * np.sin(th) + rr ** 3 / 3 * np.sin(3 * th))
zz = a * ((rr ** 2) * np.cos(2 * th))

ax.plot_surface(xx, yy, zz, edgecolor='black', facecolor='purple', alpha=0.7, linewidth=1, rcount=500, ccount=500)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.tight_layout()
plt.show()

0. 7.0 stitches in a magic loop
1. +9.0 (16.0)
2. +12.0 (28.0)
3. +13.0 (41.0)
4. +13.0 (54.0)
5. +15.0 (69.0)
6. +15.0 (84.0)
7. +16.0 (100)

Divide into four sections all with 25 stitches

Number of stitches per inner section
    5    9   12
Number of stitches moved in
    4    3    2
Increases per inner section
    1    1    1

Number of stitches per outer section
   24   24   25
Increases per outer section
    3    3    3

Number of stitches in the model 795
